# Data Preprocessing

1. **Data Loading and Preprocessing**:
   - Loads the data from a CSV file and standardizes specific columns (such as loan information, user states, etc.) for model training.
   - Creates a new feature `installment_timestep` based on `loan_id` and `installment`.

2. **Data Splitting**:
   - Splits the data into training (`train`) and testing (`test`) sets based on the `sample` and `group` columns.

3. **Feature and Label Preparation**:
   - For each `loan_id`, extracts features and labels.
   - Features include `loan_id`, user states, loan information, etc.
   - Labels correspond to the next time step of the relevant state variables (e.g., prediction of `installment`).

4. **Data Saving and Batching**:
   - Saves the processed data as CSV files and stores the data in multiple batches by `loan_id` into pickle files for later training.

5. **Training and Validation Split**:
   - Randomly selects 10% of the training data as a validation set and the rest as the training set.

The ultimate goal of this code is to save the processed data in a format suitable for training, ensuring that the data is standardized, properly split, and stored.

In [1]:
import pandas as pd
data = pd.read_csv('./Data/20240205fullsample_new.csv')
data.head()

,group,id,loan_id,EID,installment,action,action_num_should,action_num_actual,cumu_action0,cumu_action1,...,reward_recovery_rate_nn13,reward_recovery_rate_nn14,delay_due_days,delay_due_days_diff,recovery_rate_total,loan_done,installment_timestep,recovery_rate_weights,recovery_rate_weighted,sample
0,train,51,/+6C2lDDYJgzzCXpn96AFA==,4,1,no_action,0,0,0,0,...,0.033353,0.034837,0,0,0.000000,0,1,1.062157,0.205350,rlsimulator
1,train,53,/+6C2lDDYJgzzCXpn96AFA==,4,2,self,1,1,1,0,...,0.052457,0.031081,2,2,0.000000,0,1,0.868746,0.000000,rlsimulator
2,train,54,/+6C2lDDYJgzzCXpn96AFA==,4,2,family,2,2,1,1,...,0.052457,0.019864,7,5,0.000000,0,2,0.868746,0.000000,rlsimulator
3,train,55,/+6C2lDDYJgzzCXpn96AFA==,4,2,acquiantance,3,3,1,1,...,0.088502,0.045893,7,0,0.000000,0,3,0.868746,0.000000,rlsimulator
4,train,56,/+6C2lDDYJgzzCXpn96AFA==,4,2,sms,4,4,1,1,...,0.088502,0.097007,8,1,0.185833,0,4,0.868746,0.177079,rlsimulator


In [2]:
# show data summary
data.describe()

,id,EID,installment,action_num_should,action_num_actual,cumu_action0,cumu_action1,cumu_action2,cumu_action3,cumu_action4,...,reward_recovery_rate_nn,reward_recovery_rate_nn13,reward_recovery_rate_nn14,delay_due_days,delay_due_days_diff,recovery_rate_total,loan_done,installment_timestep,recovery_rate_weights,recovery_rate_weighted
count,223349.000000,223349.000000,223349.000000,223349.000000,223349.000000,223349.000000,223349.000000,223349.000000,223349.000000,223349.000000,...,175098.000000,175098.000000,175098.000000,223349.000000,223349.000000,223349.000000,223349.000000,223349.000000,180745.000000,223349.000000
mean,107706.616819,40.165181,3.778853,1.935397,1.226878,2.745927,2.432449,1.777451,1.160377,0.636605,...,0.038105,0.041555,0.037951,6.362249,2.482630,0.026892,0.069595,2.130128,0.759736,0.041230
std,76169.461842,22.401824,1.935105,1.381049,1.306034,2.613160,1.986780,1.701290,1.352796,0.960960,...,0.030533,0.018976,0.029933,6.057941,2.229151,0.092568,0.254464,1.144701,0.507806,0.082932
min,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,-0.038912,-0.111341,-0.060959,0.000000,0.000000,0.000000,0.000000,1.000000,0.006570,0.000000
25%,39523.000000,21.000000,2.000000,1.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,...,0.021095,0.030848,0.020862,1.000000,0.000000,0.000000,0.000000,1.000000,0.456610,0.000000
50%,94941.000000,40.000000,4.000000,2.000000,1.000000,2.000000,2.000000,1.000000,1.000000,0.000000,...,0.033013,0.039296,0.032354,5.000000,3.000000,0.000000,0.000000,2.000000,0.826169,0.000000
75%,174269.000000,60.000000,5.000000,3.000000,2.000000,4.000000,4.000000,3.000000,2.000000,1.000000,...,0.046915,0.048261,0.045947,11.000000,3.000000,0.000000,0.000000,3.000000,0.990000,0.000000
max,253457.000000,78.000000,10.000000,4.000000,4.000000,23.000000,10.000000,10.000000,10.000000,8.000000,...,0.334280,0.172475,0.349052,64.000000,59.000000,1.870000,1.000000,4.000000,20.391145,1.692020


In [ ]:
# show data columns names
data.columns

In [ ]:
data['installment_timestep'] = data.groupby(
    ['loan_id', 'installment']).cumcount()+1

In [ ]:
states = ['installment', 'installment_timestep', 'state_cum_overduelength',
          'remaining_debt', 'state_capital', 'state_interests',
          'state_penalty', 'gender', 'age',
          'amount', 'num_loan', 'duration',
          'year_ratio', 'diff_city', 'marriage',
          'kids', 'month_in', 'housing',
          'edu', 'motivation']

data[states] = (data[states] - data[states].mean()) / data[states].std()

In [ ]:
data

In [ ]:
data_rlsim = data[data['sample'] == 'rlsimulator']
data_rlsim.shape

In [ ]:
train = data.loc[(data['sample'] == 'rlsimulator')
                 & (data['group'] == 'train')]
train.shape

In [ ]:
test = data.loc[(data['sample'] == 'rlsimulator')
                & (data['group'] == 'test')]
test.shape

In [ ]:
data_list = [train, test]
data_name_list = ['train', 'test']
# data_list = [test]
# data_name_list = ['test']

In [ ]:
from tqdm.auto import tqdm
import os


for j in tqdm(range(len(data_list)), leave=True):
    dt = data_list[j]
    dt_name = data_name_list[j]  # Get the name of the current data set
    dt_loan_ids = dt['loan_id'].drop_duplicates().tolist()

    X_df = pd.DataFrame()
    y_df = pd.DataFrame()

    for loan_id in tqdm(dt_loan_ids, leave=True):
        df1 = dt.loc[dt['loan_id'] == loan_id]

        X_train = df1[['loan_id'] + states + ['action_num_actual',
                                              'installment_done',
                                              'loan_done',
                                              'recovery_rate_weighted']]
        X_df = pd.concat([X_df, X_train], ignore_index=True)

        y_train: pd.DataFrame = df1[['installment', 'installment_timestep', 'state_cum_overduelength',
                                    'remaining_debt', 'state_capital', 'state_interests', 'state_penalty']]

        y_train = y_train.rename(columns={'installment': 'installment.1',
                                          'installment_timestep': 'installment_timestep.1',
                                          'state_cum_overduelength': 'state_cum_overduelength.1',
                                          'remaining_debt': 'remaining_debt.1',
                                          'state_capital': 'state_capital.1',
                                          'state_interests': 'state_interests.1',
                                          'state_penalty': 'state_penalty.1'})

        if y_train.shape[0] > 1:
            y_train = y_train[1:]
            y_train = pd.concat(
                [y_train, y_train.iloc[[-1]]], ignore_index=True)

        y_df = pd.concat([y_df, y_train], ignore_index=True)

    # save the data set
    save_folder = f'./Res/Data/{dt_name}'  # create different save directories based on the dataset name
    if not os.path.exists(save_folder):
        os.makedirs(save_folder)

    X_df.to_csv(f'{save_folder}/X_df.csv', index=False)
    y_df.to_csv(f'{save_folder}/y_df.csv', index=False)

In [ ]:
import pandas as pd
# test the saved data
x_df_train = pd.read_csv('./Res/Data/train/X_df.csv')
y_df_train = pd.read_csv('./Res/Data/train/y_df.csv')

X_df_test = pd.read_csv('./Res/Data/test/X_df.csv')
y_df_test = pd.read_csv('./Res/Data/test/y_df.csv')

In [ ]:
train_combine = pd.concat([x_df_train, y_df_train], axis=1)
train_combine.to_csv('./Res/Data/train/train_combine.csv', index=False)
train_combine.to_excel('./Res/Data/train/train_combine.xlsx', index=False, engine='openpyxl')

In [ ]:
test_combine = pd.concat([X_df_test, y_df_test], axis=1)
test_combine.to_csv('./Res/Data/test/test_combine.csv', index=False)
test_combine.to_excel('./Res/Data/test/test_combine.xlsx', index=False, engine='openpyxl')